In [24]:
import numpy as np
from tqdm import tqdm
from numba import njit
import pandas as pd
# from project import Project

In [27]:
# dataSet = pd.read_csv('card_transdata.csv')
dataSet = pd.read_csv('Book1.csv', header=None)

## Remove missing data

In [7]:
# dataSet.dropna(inplace=True)

## Handle Outliers

In [28]:
#Handle outliers
def handle_outliers(df):
    # Select numerical columns only
    num_cols = df.select_dtypes(include=[np.number])
    
    # Compute the 1st and 99th percentile of each numerical column
    percentiles = np.nanpercentile(num_cols, [1, 99], axis=0)

    # Winsorize the numerical columns
    num_cols = np.clip(num_cols, percentiles[0], percentiles[1])

    # Replace the original numerical columns in the dataframe with the winsorized ones
    df[num_cols.columns] = num_cols

# handle_outliers(dataSet)

# for col in NUM_COL:
#     plotting(dataSet, col)

# dataSet.describe()
(dataSet.head(5))


,0,1,2,3,4
0,A,B,NaN,NaN,NaN
1,NaN,B,NaN,D,NaN
2,NaN,B,C,NaN,NaN
3,A,B,NaN,D,NaN
4,A,NaN,C,NaN,NaN


In [9]:
##convert the distance from home where >40 is Far and 15<distance<40 is Medium and <15 is Close
#use function cut

# dataSet['distance_from_home'] = pd.qcut(dataSet['distance_from_home'], q=3, labels=['Close_from_home', 'Medium_from_home', 'Far_from_home'])
# dataSet['distance_from_last_transaction'] = pd.qcut(dataSet['distance_from_last_transaction'], q=3, labels=['Close_from_lt', 'Medium_from_lt', 'Far_from_lt'])
# dataSet['ratio_to_median_purchase_price'] = pd.qcut(dataSet['ratio_to_median_purchase_price'], q=4, labels=['Low_ratio', 'Medium_ratio', 'High_ratio','Extreme_ratio'])
# dataSet['repeat_retailer'] = pd.cut(dataSet['repeat_retailer'], bins=[-0.5, 0.9, np.inf], labels=['no_repeat', 'repeat'])
# dataSet['used_chip'] = pd.cut(dataSet['used_chip'], bins=[-0.5, 0.9, np.inf], labels=['no_chip', 'chip'])
# dataSet['used_pin_number'] = pd.cut(dataSet['used_pin_number'], bins=[-0.5, 0.9, np.inf], labels=['no_pin', 'pin'])
# dataSet['online_order'] = pd.cut(dataSet['online_order'], bins=[-0.5, 0.9, np.inf], labels=['offline', 'online'])
# dataSet['fraud'] = pd.cut(dataSet['fraud'], bins=[-0.5, 0.9, np.inf], labels=['not_fraud', 'fraud'])
# (dataSet.head(5))


,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order,fraud
0,Far_from_home,Close_from_lt,High_ratio,repeat,chip,no_pin,offline,not_fraud
1,Medium_from_home,Close_from_lt,High_ratio,repeat,no_chip,no_pin,offline,not_fraud
2,Close_from_home,Medium_from_lt,Low_ratio,repeat,no_chip,no_pin,online,not_fraud
3,Close_from_home,Far_from_lt,Low_ratio,repeat,chip,no_pin,online,not_fraud
4,Far_from_home,Medium_from_lt,Extreme_ratio,repeat,chip,no_pin,online,not_fraud


In [29]:
# Convert the dataframe into a list of transactions
transactions = []
for i in tqdm(range(len(dataSet))):
    transactions.append([str(item) for item in dataSet.iloc[i]])
    
print(transactions[0:5])

# Set the minimum support and confidence thresholds
min_support = 2/9
min_confidence = 0.5





100%|██████████| 9/9 [00:00<00:00, 9220.50it/s]

[['A', 'B', 'nan', 'nan', 'nan'], ['nan', 'B', 'nan', 'D', 'nan'], ['nan', 'B', 'C', 'nan', 'nan'], ['A', 'B', 'nan', 'D', 'nan'], ['A', 'nan', 'C', 'nan', 'nan']]


In [85]:
# Generate a list of frequent 1-itemsets
def generate_frequent_1_itemsets(transactions, min_support):
    #key = item, value = count
    item_counts = {}
    frequent_items = []
    for transaction in transactions:
        for item in transaction:
            if item in item_counts:
                item_counts[item] += 1
            else:
                item_counts[item] = 1
    
    for item, count in item_counts.items():
        if item == 'nan':
            continue
        support = count / len(transactions)
        if support >= min_support:
            frequent_items.append(set(item))
    return frequent_items
frequent_itemsets = generate_frequent_1_itemsets(transactions, min_support)

print((frequent_itemsets))


[{'A'}, {'B'}, {'D'}, {'C'}]


In [86]:
# Generate frequent k-itemsets
from itertools import combinations
k = 2

while len(frequent_itemsets) > 0:
    candidate_itemsets = []
    # Generate candidate itemsets of size k
    for i in range(len(frequent_itemsets)):
        for j in range(i+1, len(frequent_itemsets)):
            itemset1 = set(frequent_itemsets[i])
            itemset2 = set(frequent_itemsets[j])
            print("itemset1", itemset1, "itemset2", itemset2)
            candidate_itemset = list(itemset1.union(itemset2))
            # Prune the candidate itemsets
            if all([set(itemset) in frequent_itemsets for itemset in combinations(candidate_itemset, k-1)]):
                candidate_itemsets.append(candidate_itemset)
            # print(frequent_itemsets)

                # if itemset in frequent_itemsets:
    
                    
    # Count the support of each candidate itemset
    item_counts = {}
    for transaction in transactions:
        for candidate_itemset in candidate_itemsets:
            if set(candidate_itemset).issubset(set(transaction)):
                if str(candidate_itemset) in item_counts:
                    item_counts[str(candidate_itemset)] += 1
                else:
                    item_counts[str(candidate_itemset)] = 1
    # Generate a list of frequent k-itemsets
    frequent_itemsets = []
    for itemset, count in item_counts.items():
        support = count / len(transactions)
        if support >= min_support:
            # print("itemset", itemset)
            tempSet = set()
            for item in eval(itemset):
                tempSet.add(item)
            frequent_itemsets.append(tempSet)
    
    print("frequent_itemsets", frequent_itemsets)
    k += 1

itemset1 {'A'} itemset2 {'B'}
itemset1 {'A'} itemset2 {'D'}
itemset1 {'A'} itemset2 {'C'}
itemset1 {'B'} itemset2 {'D'}
itemset1 {'B'} itemset2 {'C'}
itemset1 {'D'} itemset2 {'C'}
frequent_itemsets [{'B', 'A'}, {'B', 'D'}, {'B', 'C'}, {'A', 'C'}]
itemset1 {'B', 'A'} itemset2 {'B', 'D'}
itemset1 {'B', 'A'} itemset2 {'B', 'C'}
itemset1 {'B', 'A'} itemset2 {'A', 'C'}
itemset1 {'B', 'D'} itemset2 {'B', 'C'}
itemset1 {'B', 'D'} itemset2 {'A', 'C'}
itemset1 {'B', 'C'} itemset2 {'A', 'C'}
frequent_itemsets [{'B', 'A', 'C'}, {'B', 'C', 'A'}]
itemset1 {'B', 'A', 'C'} itemset2 {'B', 'C', 'A'}
frequent_itemsets [{'B', 'C', 'A'}]
frequent_itemsets []


In [66]:
print(frequent_itemsets)
print(candidate_itemsets)

[]
[]
